# Distributed debate

This example simulates a debate competition with three participant agents, including the affirmative side (Pro), the negative side (Con), and the adjudicator (Judge). 

Pro believes that AGI can be achieved using the GPT model framework, while Con contests it. Judge listens to both sides' arguments and provides an analytical judgment on which side presented a more compelling and reasonable case.

Messages generated by any agents can be observed by other agents in the debate.

The full codes can be found in `examples/distributed/distributed_debate.py`

To install AgentScope, please follow the steps in [README.md](https://github.com/alibaba/AgentScope/blob/main/README.md#installation).

In this case, we need to initialize three agents. The codes are the follows.

```bash
def setup_server(parsed_args: argparse.Namespace) -> None:
    """Setup rpc server for participant agent"""
    agentscope.init(
        model_configs="configs/model_configs.json",
    )
    with open(
        "configs/debate_agent_configs.json",
        "r",
        encoding="utf-8",
    ) as f:
        configs = json.load(f)
        config = configs[parsed_args.role]
        host = getattr(parsed_args, f"{parsed_args.role}_host")
        port = getattr(parsed_args, f"{parsed_args.role}_port")
        server_launcher = RpcAgentServerLauncher(
            host=host,
            port=port,
            local_mode=False,
            agent_class=RpcDialogAgent,
            **config,
        )
        server_launcher.launch()
        server_launcher.wait_until_terminate()
```

To make it running on jupyter, we run them in the backend.

In [1]:
%cd ../examples/distributed/

/Users/qianbingchen/AgentScope/examples/distributed


/Users/qianbingchen/miniconda3/envs/agent/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import subprocess

# Using Popen to initialize a backstage process

proc_pro = subprocess.Popen(
    ['python', 'distributed_debate.py', '--role', 'pro', '--pro-host', 'localhost', '--pro-port', '12011'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

proc_con = subprocess.Popen(
    ['python', 'distributed_debate.py', '--role', 'con', '--con-host', 'localhost', '--con-port', '12012'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

proc_judge = subprocess.Popen(
    ['python', 'distributed_debate.py', '--role', 'judge', '--judge-host', 'localhost', '--judge-port', '12013'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# The above codes will return immediately and the process will run in the background


Now, we turn back to run the main function.

In [4]:
%cd ../../notebook/

/Users/qianbingchen/AgentScope/notebook


/Users/qianbingchen/miniconda3/envs/agent/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
model_configs = [
    {
        "type": "openai",
        "name": "gpt-3.5-turbo",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    },
    {
        "type": "openai",
        "name": "gpt-4",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    }
]

In [ ]:
# -*- coding: utf-8 -*-
""" An example of distributed debate """

import argparse
import json

import agentscope
from agentscope.msghub import msghub
from agentscope.agents.rpc_dialog_agent import RpcDialogAgent
from agentscope.message import Msg
from agentscope.utils.logging_utils import logger

ANNOUNCEMENT = """
Welcome to the debate on whether Artificial General Intelligence (AGI) can be achieved using the GPT model framework. This debate will consist of three rounds. In each round, the affirmative side will present their argument first, followed by the negative side. After both sides have presented, the adjudicator will summarize the key points and analyze the strengths of the arguments.

The rules are as follows:

Each side must present clear, concise arguments backed by evidence and logical reasoning.
No side may interrupt the other while they are presenting their case.
After both sides have presented, the adjudicator will have time to deliberate and will then provide a summary, highlighting the most persuasive points from both sides.
The adjudicator's summary will not declare a winner for the individual rounds but will focus on the quality and persuasiveness of the arguments.
At the conclusion of the three rounds, the adjudicator will declare the overall winner based on which side won two out of the three rounds, considering the consistency and strength of the arguments throughout the debate.
Let us begin the first round. The affirmative side: please present your argument for why AGI can be achieved using the GPT model framework.
"""  # noqa


"""Setup the main debate competition process"""

agentscope.init(
    model_configs=model_configs,
)
pro_agent = RpcDialogAgent(
    name="Pro",
    host="localhost",
    port=12011,
    launch_server=False,
)
con_agent = RpcDialogAgent(
    name="Con",
    host="localhost",
    port=12012,
    launch_server=False,
)
judge_agent = RpcDialogAgent(
    name="Judge",
    host="localhost",
    port=12013,
    launch_server=False,
)
participants = [pro_agent, con_agent, judge_agent]
hint = Msg(name="System", content=ANNOUNCEMENT)
x = None
with msghub(participants=participants, announcement=hint):
    for _ in range(3):
        pro_resp = pro_agent(x)
        logger.chat(pro_resp.update_value())
        con_resp = con_agent(pro_resp)
        logger.chat(con_resp.update_value())
        x = judge_agent(con_resp)
        logger.chat(x.update_value())
    x = judge_agent(x)
    logger.chat(x.update_value())


Finally, terminte process in the backend.

In [ ]:
proc_pro.terminate()
proc_con.terminate()
proc_judge.terminate()